In [1]:
def data_processing(sample, ticker, base_date):
    sample.reset_index(inplace=True)
    sample.reset_index(inplace=True, drop=True)
    sample['CODE'] = ticker
    sample = sample[sample['Date'] >= base_date][['Date', 'CODE','Close']].copy()
    sample.reset_index(inplace=True, drop=True)
    sample['STD_YM'] = sample['Date'].map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
    sample['1M_RET'] = 0.0
    ym_keys = list(sample['STD_YM'].unique())
    return sample, ym_keys

def data_preprocessing(read_df, base_date):   
    read_df = read_df[read_df['Date'] >= base_date].copy()
    read_df.reset_index(inplace=True, drop = True)
    read_df['STD_YM'] = read_df['Date'].map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
    read_df['1M_RET'] = 0.0
    ym_keys = list(read_df['STD_YM'].unique())
    return read_df, ym_keys

def create_trade_book(sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes].copy()
    book['STD_YM'] = book.index.map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
    for c in sample_codes:
        book['p '+c] = ''
        book['r '+c] = ''
    return book

# 상대모멘텀 Positioning
def positions(book, s_codes):
    std_ym = ''
    buy_phase = False
    for s in s_codes : 
        print(s)
        for i in book.index:
            if book.loc[i,'p '+s] == '' and book.shift(1).loc[i,'p '+s] == 'ready ' + s:
                std_ym = book.loc[i,'STD_YM']
                buy_phase = True
            
            if book.loc[i,'p '+s] == '' and book.loc[i,'STD_YM'] == std_ym and buy_phase == True : 
                book.loc[i,'p '+s] = 'buy ' + s
            
            if book.loc[i,'p '+ s] == '' :
                std_ym = None
                buy_phase = False
    return book

def multi_returns(book, s_codes):
    # 손익 계산
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}
    
    for i in book.index:
        for s in s_codes:
            if book.loc[i, 'p ' + s] == 'buy '+ s and \
            book.shift(1).loc[i, 'p '+s] == 'ready '+s and \
            book.shift(2).loc[i, 'p '+s] == '' :     # long 진입
                buy_dict[s] = book.loc[i, s]
            elif book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i, 'p '+s] == 'buy '+ s:     # long 청산
                sell_dict[s] = book.loc[i, s]
                # 손익 계산
                rtn = (sell_dict[s] / buy_dict[s]) -1
                book.loc[i, 'r '+s] = rtn
                print('개별 청산일 : ',i,' 종목코드 : ', s , 'long 진입가격 : ', buy_dict[s], ' |  long 청산가격 : ', sell_dict[s], ' | return:', round(rtn * 100, 2),'%') # 수익률 계산.

            if book.loc[i, 'p '+ s] == '':     # zero position || long 청산.
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0


    acc_rtn = 1.0        
    for i in book.index:
        rtn  = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s:  # 청산. 이떄 수익률이 나오니깐.
                count += 1
                rtn += book.loc[i, 'r '+s]
        if (rtn != 0.0) & (count != 0) :
            # 내가 후보로 삼은 universe에서  1/n 이라고 가정.
            acc_rtn *= (rtn /count )  + 1
            print('누적 청산일 : ',i,'청산 종목수 : ',count, \
                  '청산 수익률 : ',round((rtn /count),4),'누적 수익률 : ' ,round(acc_rtn, 4)) # 수익률 계산.
        book.loc[i,'acc_rtn'] = acc_rtn
    
    print ('누적 수익률 :', round(acc_rtn, 4))

In [2]:
import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr

month_last_df = pd.DataFrame(columns=['Date', 'CODE', '1M_RET'])
stocks = pd.read_csv('./stock_codes.csv')
stocks['code'] = stocks['code'].map(lambda x : x[1:])
stocks.set_index('code', inplace=True)
print(stocks)

              label
code               
005930  SAMSUNGelec
051910       LGchem
000660      SKhynix
207940   SAMSUNGbio
035420        NAVER
035720        KAKAO
005380    HYNDAIcar
006400   SAMSUNGsdi
068270    CELLTRION
000270          KIA


In [3]:
dataframes = []
month_last_df = pd.DataFrame()
read_df = pd.DataFrame()
for st in stocks.index:
    temp_df = fdr.DataReader(st)
    temp_df.reset_index(inplace=True)
    temp_df['CODE'] = stocks.loc[st,'label']
    read_df = read_df.append(temp_df.loc[:])
    print(temp_df)
    
    #month_last_df = month_last_df.append(temp_df.loc[:, ['Date', 'CODE', 'Close','1M_RET']])

           Date   Open   High    Low  Close    Volume    Change         CODE
0    1997-09-01   1258   1268   1236   1251     76170       NaN  SAMSUNGelec
1    1997-09-02   1238   1268   1227   1269     97370  0.014388  SAMSUNGelec
2    1997-09-03   1268   1268   1236   1237    108600 -0.025217  SAMSUNGelec
3    1997-09-04   1247   1247   1217   1221     88900 -0.012935  SAMSUNGelec
4    1997-09-05   1220   1250   1217   1244     38970  0.018837  SAMSUNGelec
...         ...    ...    ...    ...    ...       ...       ...          ...
5995 2021-09-13  75200  76300  75100  76300  11397775  0.013280  SAMSUNGelec
5996 2021-09-14  77100  77700  76600  76600  18167057  0.003932  SAMSUNGelec
5997 2021-09-15  77400  77400  76400  77000  12829128  0.005222  SAMSUNGelec
5998 2021-09-16  77300  77400  76100  76100  13067622 -0.011688  SAMSUNGelec
5999 2021-09-17  76300  77200  75900  77200  15656562  0.014455  SAMSUNGelec

[6000 rows x 8 columns]
           Date    Open    High     Low   Close   V

           Date    Open    High     Low   Close   Volume    Change CODE
0    1997-09-01  100971  108873   99215  106239    78070       NaN  KIA
1    1997-09-02  110629  110629  107117  109751    12600  0.033058  KIA
2    1997-09-03  111507  111507  107995  109751    24070  0.000000  KIA
3    1997-09-04  107995  109751  101849  108873    33280 -0.008000  KIA
4    1997-09-05  101849  105361  100971  104483   222350 -0.040322  KIA
...         ...     ...     ...     ...     ...      ...       ...  ...
5995 2021-09-13   82300   83100   81800   82600  1367010  0.001212  KIA
5996 2021-09-14   83300   84500   83100   83400  1530277  0.009685  KIA
5997 2021-09-15   83000   84200   82500   83300  1124683 -0.001199  KIA
5998 2021-09-16   83600   84500   83400   84500  1424238  0.014406  KIA
5999 2021-09-17   84300   84700   83700   84100  1417299 -0.004734  KIA

[6000 rows x 8 columns]


In [8]:
month_last_df = pd.DataFrame(columns=['Date','CODE','Close'])
month_last_ret_df = pd.DataFrame(columns=['Date','CODE','1M_RET'])
price_df, ym_keys = data_preprocessing(read_df, base_date='2010-01-02')
read_df

,Date,Open,High,Low,Close,Volume,Change,CODE
0,1997-09-01,1258,1268,1236,1251,76170,NaN,SAMSUNGelec
1,1997-09-02,1238,1268,1227,1269,97370,0.014388,SAMSUNGelec
2,1997-09-03,1268,1268,1236,1237,108600,-0.025217,SAMSUNGelec
3,1997-09-04,1247,1247,1217,1221,88900,-0.012935,SAMSUNGelec
4,1997-09-05,1220,1250,1217,1244,38970,0.018837,SAMSUNGelec
...,...,...,...,...,...,...,...,...
5995,2021-09-13,82300,83100,81800,82600,1367010,0.001212,KIA
5996,2021-09-14,83300,84500,83100,83400,1530277,0.009685,KIA
5997,2021-09-15,83000,84200,82500,83300,1124683,-0.001199,KIA
5998,2021-09-16,83600,84500,83400,84500,1424238,0.014406,KIA


In [9]:

s_codes = list(read_df['CODE'].unique())

for s in s_codes:
    print(s)
    for ym in ym_keys:
        try : 
            # 절대 모멘텀 사용 종가
            month_last_df= month_last_df.append(price_df.loc[price_df[(price_df['STD_YM'] == ym) \
                                                                      & (price_df['CODE'] == s)].index[-1]\
                                                             ,['Date','CODE','Close']\
                                                            ]
                                               )
            
            # 상대 모멘텀 사용 수익률
            m_ret = price_df.loc[price_df[(price_df['STD_YM'] == ym) & (price_df['CODE'] == s)].index[-1] , 'Close'] \
            / price_df.loc[price_df[(price_df['STD_YM'] == ym) & (price_df['CODE'] == s)].index[0] , 'Close']
            price_df.loc[(price_df['STD_YM'] == ym) & (price_df['CODE'] == s), ['1M_RET']] = m_ret
            
            
            month_last_ret_df= month_last_ret_df.append(price_df.loc[price_df[(price_df['STD_YM'] == ym) \
                                                                      & (price_df['CODE'] == s)].index[-1]\
                                                             ,['Date','CODE','1M_RET']\
                                                            ]
                                               )
        except IndexError:
            pass

SAMSUNGelec
LGchem
SKhynix
SAMSUNGbio
NAVER
KAKAO
HYNDAIcar
SAMSUNGsdi
CELLTRION
KIA


In [10]:
# 12개월  절대 모멘텀 
bf_month = 12
month_last_df.reset_index(inplace=True,drop=True)
month_last_tmp_df = month_last_df.pivot('Date','CODE','Close').copy()
abs_momentum_filter =  month_last_tmp_df / month_last_tmp_df.shift(bf_month) -1
abs_momentum_filter = abs_momentum_filter.where(abs_momentum_filter > 0.0 , np.nan)
abs_momentum_filter.fillna(0,inplace=True)
abs_momentum_filter[abs_momentum_filter != 0] = 1


# 유니버스 내 상위 20% 상대 모멘텀
ratio = 0.2   # 상위 20%에 드는 종목들만.
month_last_ret_df.reset_index(inplace=True,drop=True)
month_last_ret_df = month_last_ret_df.pivot('Date','CODE','1M_RET').copy()
rel_momentum_filter = month_last_ret_df.rank(axis=1, ascending=False, method="max", pct=True) # 투자종목 선택할 rank
rel_momentum_filter = rel_momentum_filter.where( rel_momentum_filter < ratio , np.nan)
rel_momentum_filter.fillna(0,inplace=True)
rel_momentum_filter[rel_momentum_filter != 0] = 1

#듀얼 모멘텀.
dual_mementum_filter = abs_momentum_filter * rel_momentum_filter # 절대 모멘텀 * 상대 모멘텀 => 듀얼 모멘텀

In [11]:

# 사실상 trading 부분.
sig_dict = dict()
for date in dual_mementum_filter.index:
    ticker_list = list(dual_mementum_filter.loc[date,dual_mementum_filter.loc[date,:] >= 1.0].index)
    sig_dict[date] = ticker_list
    
stock_c_matrix = price_df.pivot('Date','CODE','Close').copy()
book = create_trade_book(stock_c_matrix, s_codes)
book['STD_YM'] = book.index.map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))

#ready 표시
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date,'p '+ stock] = 'ready ' + stock
        

# # trading 부분을 만들려고 하는데
book = positions(book, s_codes)

SAMSUNGelec
LGchem
SKhynix
SAMSUNGbio
NAVER
KAKAO
HYNDAIcar
SAMSUNGsdi
CELLTRION
KIA


In [12]:
multi_returns(book, s_codes)

개별 청산일 :  2011-03-02 00:00:00  종목코드 :  SKhynix long 진입가격 :  29650.0  |  long 청산가격 :  27550.0  | return: -7.08 %
개별 청산일 :  2011-04-01 00:00:00  종목코드 :  KAKAO long 진입가격 :  19228.0  |  long 청산가격 :  19931.0  | return: 3.66 %
개별 청산일 :  2011-05-02 00:00:00  종목코드 :  LGchem long 진입가격 :  458000.0  |  long 청산가격 :  540000.0  | return: 17.9 %
개별 청산일 :  2011-06-01 00:00:00  종목코드 :  HYNDAIcar long 진입가격 :  254500.0  |  long 청산가격 :  243500.0  | return: -4.32 %
개별 청산일 :  2011-08-01 00:00:00  종목코드 :  CELLTRION long 진입가격 :  30106.0  |  long 청산가격 :  40519.0  | return: 34.59 %
개별 청산일 :  2011-09-01 00:00:00  종목코드 :  KAKAO long 진입가격 :  26775.0  |  long 청산가격 :  25691.0  | return: -4.05 %
개별 청산일 :  2011-10-04 00:00:00  종목코드 :  NAVER long 진입가격 :  65134.0  |  long 청산가격 :  71256.0  | return: 9.4 %
개별 청산일 :  2011-11-01 00:00:00  종목코드 :  KAKAO long 진입가격 :  26896.0  |  long 청산가격 :  26133.0  | return: -2.84 %
개별 청산일 :  2011-12-01 00:00:00  종목코드 :  LGchem long 진입가격 :  365500.0  |  long 청산가격 :  346500.0  | return: -5.2

개별 청산일 :  2021-05-03 00:00:00  종목코드 :  CELLTRION long 진입가격 :  324500.0  |  long 청산가격 :  249500.0  | return: -23.11 %
개별 청산일 :  2021-06-01 00:00:00  종목코드 :  LGchem long 진입가격 :  907000.0  |  long 청산가격 :  825000.0  | return: -9.04 %
개별 청산일 :  2021-07-01 00:00:00  종목코드 :  CELLTRION long 진입가격 :  272500.0  |  long 청산가격 :  267500.0  | return: -1.83 %
개별 청산일 :  2021-08-02 00:00:00  종목코드 :  KAKAO long 진입가격 :  160000.0  |  long 청산가격 :  146000.0  | return: -8.75 %
개별 청산일 :  2021-09-01 00:00:00  종목코드 :  NAVER long 진입가격 :  433500.0  |  long 청산가격 :  445000.0  | return: 2.65 %
누적 청산일 :  2011-03-02 00:00:00 청산 종목수 :  1 청산 수익률 :  -0.0708 누적 수익률 :  0.9292
누적 청산일 :  2011-04-01 00:00:00 청산 종목수 :  1 청산 수익률 :  0.0366 누적 수익률 :  0.9631
누적 청산일 :  2011-05-02 00:00:00 청산 종목수 :  1 청산 수익률 :  0.179 누적 수익률 :  1.1356
누적 청산일 :  2011-06-01 00:00:00 청산 종목수 :  1 청산 수익률 :  -0.0432 누적 수익률 :  1.0865
누적 청산일 :  2011-08-01 00:00:00 청산 종목수 :  1 청산 수익률 :  0.3459 누적 수익률 :  1.4623
누적 청산일 :  2011-09-01 00:00:00 청산 종목수 :  1 청산 수익률 :